# Sequence Loading into a Keysight M8190A Arbitray Waveform Generator.

In this notebook we will explain how to load rectangular pulse schemes sequences into the Arbitrary Waveform Generator (AWG) using the following python modules:

1. Sweeping.py
2. PulseFiles.py
3. Dictionaries.py
4. Instrument.py

As well as standard python modules used for Scientific Analysis (numpy, pandas, time and matplotib),PyVisa ,and the National Instruments API for controlling a  Data Acquisition (DAQ) Box.

This Notebook is divided in 3 sections:

1. __Initialization__ where we se how to connect to the AWG through Python and use the proper settings to prepare the AWG into playing Waveform data.

2. __Pulse File Information__ where we explain the details about the waveform data and how it can be loaded into the AWG.

3. __DAQ Measurement__ how we can set up the AWG into playing a sequence as well as capturing data with the DAQ box.


All of the instrument settings metioned here will refer to the 14 bit mode and the Output Channel 1.

The version of this notebook is 1.00

# Initialization

## First, as usual, we import the required Python Modules:

In [ ]:
import numpy as np
from numpy import loadtxt
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
import pyvisa as visa
from matplotlib import animation
from time import sleep
import pyqtgraph as pg 
import nidaqmx
import pickle


#Importing the functions from the module and the pulses dictionaries
from Sweeping import *
from Dictionaries import *
from PulseFiles import *
from Instrument import *


We also need to stablish the connection between the AWG and the computer. In order to do this turn the AWG on and wait until the "Access" LED turns green, afterwards run the "Keysight M8190Firmware" Software (a quick access is given in the the Desktop). Please do not confuse this with the "Keysight M8190 Soft Front Panel", this Soft Front Panel software is a graphical interphase to controll the AWG.

When the connection is stablished a window of this form will appear:

![Firmware Visa Adress](Firmware.png)


This provides us the VISA Adress needed to use the instrument through Python.

This Visa Adress, as well as other AWG parameters are stored in a dictionary, whose Keys will be used by most of the functions. 
An example of a AWG parameter Dictionary is:

In [ ]:
AWG_Settings= {
    'Visa Resource Name': 'TCPIP0::ibn3-036.ibn-net.kfa-juelich.de::hislip0::INSTR',
    'Voltage Amplitude':700, #miliVolts
    'Clock Sample Frecuency':6000000000, #Hz
    'Output_Channel': 1,
    'Mode': 'STS',
    'Trigger In Threshold' : 1, #Volts
    'Output Rout': 'DAC', #DAC output rout only has the SMA Physical outputs: Direct Out and (Averaged) Direct Out
    'Data Directory': r'D:\Alejandro\Pulses\diode measurements\Pulse Scheme CSV\Measurements\1000nano' #File path where we will save the Pulse Scheme CSV files
}

We also define a Dictionary for the DAQ settings:

In [ ]:
DAQ_Settings1 = {
    'DAQ Name' : 'DAQBNC1',
    'Analog Channel Input Marker' : 'ai2',
    'Analog Channel Input Waveform' : 'ai0 ',
    'Analog Channel Output' : 'ao1',
    'Minimum Voltage' : -10,
    'Maximum Voltage' : 10,
    'Minimum Voltage Marker' : -9,
    'Maximum Voltage Marker' : 9,
    'Maximum Current': 0.01,
    'Minimum Current' : -0.01,
    'Sampling Frequency' : 1000,
    'Data Directory': r'D:\Documentos\STM\Python Pulses\S1' #path of home computer file!
}

After this we can run the __VisaR__ function to generate the object Python class to control the AWG thorugh Python, with Pyvisa. We will name this object class as **M8190A** and choose a timeout time for the AWG of 60000 ms

In [ ]:
M8190A = VisaR(AWG_Settings,60000)

Once the object class is created we can run the __Initialization__ function, this function sets the AWG to the proper parameters and configurations in order to play pulse sequences.

In [ ]:
Initialization(M8190A,AWG_Settings)

It is important to know that this function configures the instrument into the Sequencer mode given by the AWG_Settings['Mode'] key (in the previous example it is set to STS). The AWG has 3 different sequencing modes: Arbitrary, STSequence, and STScenario.

In the Arbitrary mode, the AWG is able to play a single waveform data, named within the AWG intrphase as __Segment__. This Segments are identified within the AWG by an integer number called __Segment Id__ and will have a given __length__ , thes length defines the total number of waveform data samples to be played by the AWG. 
The minimum Segment length is 320 samples in 12 bit mode and 240 samples in 14 bit mode. And the Maximum legth is 2048 MSa

The __granularity__ of the AWG is 64 samples in 12 bit mode and 48 samples in 14 bit mode.





Segments can be combined together to from a __Sequence__ and be played through the STSequence Mode. In this regard a Sequence consistes of multiple Segments played in a given order. It is good to think of a Sequence as a Table, where each Segment is an entry. In this way, we first deifine the indiviual Segments, next we define the Sequence table and then load intoto it the Segment entries in the order that we want them to be played. 


The Advancemnt method in between Segments used here is __CONDITIONAL__ which means that the waveform of all segments in the sequence will be played indefinitely until the AWG is stoped. Other advancement methods are posible and can be view in the users Manual.

Finally, the STScneario plays , as it names suggests it a Scenario, which is a combination of multiple sequences.

# Pulse File Information

## Formating

The way that we load waveform data into the AWG Segments is through CSV files (other formats and options are mentioned in the AWG users Manual). 

This CSV files must be delimited by a comma and the sample values must be normalized ( in the [-1,1] interval, in our case, we normalize our waveform data with the AWG_Settings['Voltage Amplitude'] value). The AWG has two __Marker channels__ per Output channel and the way we enanable Marker channels to a specific waveform data is appending a value of 1 in the column to the right of the beforementioned waveform data. If we do not want to enable the marker channel to waveform data, the value to be appended is 0, instead of 1. The Marker columns can only be given the 0 or 1 values. The way we distinguish between the two marker channels is by the header of such column.

In concrete, the file itself (as the way we are implementing it) will have a total of 3 Columns labeles as:

1. Y1  (here we have the normalized waveform data values)
2. SyncMarker1   (Column of either 1 or 0, dependinding )
3. SampleMarker1 (Column of either 1 or 0, dependinding ).

The overall difference between the two Marker channels is the Sampling Frequency at which they run.

![CSV Example](csvA.png)

Image above: ehader of an example of the format of a csv file with 7500000 waveform data with the marker channels enabled.

# Number of Samples details

The total number of data samples in our sequences is not free, as it should follow the relation:

$$ N = S_r * t $$

where $N$ is the number of sample points, $S_r$ the Sampling Frequency (Sampling Clock) of the AWG in Herzs and $t$ the time regime (time length) of the desired Pulse Scheme in seconds. $N$ should also follow the next granularity Caveat.

## Granularity Caveat

As mentioned earlier, each Segment in the AWG has a given Granularity, this means that the  total number of samples in our waveform data __must__ be a multiple of this granularity (for our current settings this is __48__) In cases where the waveform data is not a multiple of the granularity, the Segment will have a length equal to: $l = k N$ where $l$ is the length of the segment, $N$ the number of samples and $k$ the lowest posible positive integer such that $l$ is a multiple of the granularity (48).



# Pulse Sequence Sweeping

Now that we know which formtat the csv files containing our waveform data should have, we will explaing how we create Pulse Sequences based on this.

The __Pulse Sequence__ that we will be loaded into the AWG will consist of two __Pulse Cycles__ (we named them A, and B correspondelty). Each Cycle has a __Pulse Scheme__ made from two pulses: Pump and Probe. Usually we will change (sweep) the realtive time position of one Pulse with respect to the other,namely we will leave the Pump pulse still while  we sweep the Probe pulse with respect to the Pump. 

![Sweeping Cycle A](CompleteSweepingSteps.jpg)


Here we show Several Sweeping Steps, for Cycle A. 

The csv files loaded into the AWG will have the proper waveform data (following the restrictions metioned earlier) of a complete Cycle (either A or B) at a given sweeping step, in other words, our waveform Segments will be pulse cycles, and our __pulse sequences__ will be the Sequence generated by the Segment of CycleA followed by the Segment of CycleB at a given sweeping step. This means advancing from a sweepin step to the next one is translated to changing AWG Sequences.

 The parameters of each individual pulse are contained in dictionaries, for example:

In [ ]:
Pump = {
    'Name':'Pump',
    'Amplitude':40, #mV, this value will be normalized in the csv file by the Voltage Amplitude key of the AWG_Settings key
    'Start time':5, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the time of the first sweeping step
    'End time':2e-10, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the time of the last sweeping step
    'Start Duration':1, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the duration of the first sweeping step
    'End Duration':1, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the duration of the last sweeping step
    'Sweep time':0, #the sweeping functions use this parameter, if it is equal to 1, the pulse will be swept in time, if it is 0 it will not. 
    'Sweep Duration':0 #the sweeping functions use this parameter, if it is equal to 1, the pulse will be swept in duration, if it is 0 it will not.
}

In [ ]:
ProbeA = {
    'Name':'ProbeA',
    'Amplitude':25, #mV, this value will be normalized in the csv file by the Voltage Amplitude key of the AWG_Settings key
    'Start time':3, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the time of the first sweeping step
    'End time':7, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the time of the last sweeping step
    'Start Duration':1, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the duration of the first sweeping step
    'End Duration':60, #the units mainly depend on the desired time regime for the pulse sequence given by the Sweeping_Single_List_File_teil function, this refers to the duration of the last sweeping step
    'Sweep time':1, #the sweeping functions use this parameter, if it is equal to 1, the pulse will be swept in time, if it is 0 it will not.
    'Sweep Duration':0 #the sweeping functions use this parameter, if it is equal to 1, the pulse will be swept in duration, if it is 0 it will not.
}

In [ ]:
ProbeB = {
    'Name':'ProbeB',
    'Amplitude':0,
    'Start time':0,
    'End time':12,
    'Start Duration':1,
    'End Duration':6,
    'Sweep time':0,
    'Sweep Duration':0
}

# NOTE
The time parameters for the pulses given here are scaled to the total pulse scheme time $t$ used in the Sweeping function, more details below. (in this example they correspond to ms)

The information of a given Pulse Scheme (Cycle) is given also in a dictionary:

In [ ]:
PulseScheme_A =  {
    'Name' : 'PulseScheme_A',
    'Pulse Scheme': [Pump,ProbeA], #the pulse scheme corresponds to a List that contains the Pulse dictionaries
    'Number of repetitions': 80000,
    'Measurement file Path': r'D:\Alejandro\Pulses\diode measurements\TList\10nano'
}

In [ ]:
PulseScheme_B  = {
    'Name' : 'PulseScheme_B',
    'Pulse Scheme': [ProbeB,ProbeB],
    'Number of repetitions': 1,
    'Measurement file Path': r'D:\Alejandro\Pulses\diode measurements\TList\1ms,50Hz'
}

The cycle waveform data at different sweeping steps is generated first as a numpy array and then exported to a CSV file with the proper formating by the function:

In [ ]:
#the parametes of this function are quickly showed as follow, for mor details please check the documentation
#Sweeping_Single_List_File_teil(PulseScheme_Dictionary,total_number_sweeping_steps,time_length_scheme,Number_Samples,Starting_sweepingstep,ending_sweepingstep,AWG_Settings,cycle)

#For a time length of:
time_length = 10#ms, 0.01s



#we know the number of samples,sam, should follow:
sam = AWG_Settings['Clock Sample Frecuency']*(time_length*0.001)
#it should aslo follow the granularity constraint



#for CycleA and 30 sweeping steps
Sweeping_runa = Sweeping_Single_List_File_teil(PulseScheme_A,30,time_length,sam,0,30,AWG_Settings,'A')[0]

#for CycleB sweeping steps
Sweeping_runa = Sweeping_Single_List_File_teil(PulseScheme_B,30,time_length,sam,0,30,AWG_Settings,'B')[0]

The above parameters of the function will give us 31 csv files,that is 31 CycleA files in which the ProbeA pulse will be swept with respect to the Pulse.

The main difference in our code between CycleA and CycleB is that CycleA have assigned markers to the waveform data on it's csv files (marker columns have a value of 1), while Cycle B does not (marker columns have a value of 0). This is mainly to modulated CycleA with the rising part of the Reference Sinusoidal Wave of a Lock-In Amplifier, while modulating Cycle B with the lowering part.

As we mentioned earilier, Our Pulse Sequence will consist of CycleA followed by CycleB.

Cycle B (for autocorrelation function measurements with an RF Diode) consists of an empy pulse scheme, with no pulses whatsoever, that also lasts the same duration of Cycle A. With the example parameters given above, this means that the total time duration of CycleA + Cycle B will be 2*time_length = 20ms , which corresponds to a 50Hz Modulation. 

One of the parameters returned by the Sweeping function is a Dictionary, whose elements are the file paths to the CSV files that it created. This way we can orderly load them into the AWG as individual sequences, having the sweeping being changing from one of them to the next, this is the one that we named in the previous example as Sweeping_runa(b) and is the one that our Sequence loading function uses.

We also export and import this dictionaries with Python's Pickle, so we can re do the measurement in the future without to explicitly write this dictionaries every time.

In [ ]:
#exporting pickle 

#pickle.dump(sweeping_run_a, open(r"{dict}\cycleA_31steps_10ms_1ms_probe_3GHz.p".format(dict = PulseScheme_A['Measurement file Path']), "wb"))  # save it into a file named sweepingA.p
#pickle.dump(sweeping_runb, open(r"{dict}\cycleB_31steps_10ms_1msprobe_3Ghz.p".format(dict = PulseScheme_A['Measurement file Path']), "wb"))  # save it into a file named sweepingB.p

In [ ]:
#importing pickle

#sweeping_run_a = pickle.load(open(r"{dict}\cycleA_31steps_10ms_1ms_probe_3GHz.p".format(dict = PulseScheme_A['Measurement file Path'])),"rb"))
#sweeping_runb =  pickle.load(open(r"{dict}\cycleB_31steps_10ms_1msprobe_3Ghz.p".format(dict = PulseScheme_A['Measurement file Path']),"rb"))

# Sweeping Run loading (for Auto Correlation measurements RF Diode)

We have a function that loads the csv files into the AWG while also controlling a DAQ box to trigger it into playing and storing the voltage output. 

Before loading sweeping runs into the AWG it is important to note that everytime that the AWG is reset or that the Sampling Frequency is changed, the AWG will take around 5 seconds to play it's first waveform after having received the play command (waveforms played afterwards will be played instantly). Therefore everytime that we first turn on the AWG or change the sampling clock we run the Dummy_File function, which loads a dummy empty Segment to the AWG, plays it for a couple of seconds and then it is stoped and deleted.

In [ ]:
Dummy_File(M8190A)

## Lock-In Dictionary

Before running the Sequence Loading function we define a Lock-In dictionary to store te values that we physically set up in the lock in, for future reference. The keys of this dictionary will be appended to the name of the measurements obtained.

In [ ]:
Lock_In_Settings = {
'Time Constant' : '300ms',
'Sensitivity' : '500 microVolts',
'Reserve' : 'Normal',
'db' : 24,
'Modulation': '50Hz'
}

Finally we come to our measurement function (Measurement_Autocorrelation_voltage), where we define a playing time which is the total time duration in which the DAQ box will be recolecting data:

In [ ]:
playing_time = 10 #in seconds

Diode_Signal, averaged_data = Measurement_Autocorrelation_voltage(M8190A,DAQ_Settings1,1000, playing_time ,sweeping_run_a,sweeping_runb,PulseScheme_A['Measurement file Path'],Lock_In_Settings)